In [1]:
from bs4 import BeautifulSoup
from requests import get
import cfscrape
import re

In [2]:
def Remove(duplicate): #remove duplicate elements in the list 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

In [7]:
scraper = cfscrape.create_scraper()  
proxies={
    'http':'http://35.169.156.54:3128',
    'https':'https://35.169.156.54:3128',
}

b_block=[] #store blocks of the most recent 100 blocks
b_blockset=set()
#input the block information(which is the same as getting each block's id)
for page in range(1,3): #include 100 blocks, since each page the website covers 50 blocks
    url= 'https://www.blockchain.com/btc/blocks?page='+str(page)
    htmltext = scraper.get(url)

    if htmltext.status_code == 200:
        soup_object = BeautifulSoup(htmltext.content, 'html.parser')
   
    flag=0
    for p in soup_object.find_all('a'):
        p1=p.get('href')
        if '/btc/block/' in p1:         
                b_blockset.add(p1.replace('/btc/block/',''))
                b_block.append(p1.replace('/btc/block/','')) #store blocks number into b_block
                b_block=Remove(b_block)

    if flag==1:
        break

In [8]:
b_sourceprice=[] #store the prices paid by senders
b_destinationprice=[] #store the prices received by receivers
b_source=[] #store each transactions' source addresses
b_destination=[] #store each transactions' destination addresses
b_pool=[] #store each block's mining pool
b_timestamp=[] #store timestamp of each transaction
b_blockfinal=[] #record all the block as the form in excel sheet
b_id=[] #record all the transaction id
for k in b_block:
    url_block= 'https://www.blockchain.com/btc/block/'+k
    htmltextblock = scraper.get(url_block)
    if htmltextblock.status_code == 200:
        soup_objectblock = BeautifulSoup(htmltextblock.content, 'html.parser')
    
    for t in soup_objectblock.find_all('span'):
        if 'Miner' in t:
            span_tag=t.parent
            next_a_tag=span_tag.findNext('a')
            mining_pool=next_a_tag.contents[0] #get each mining pool's name

        b_blocktransaction=0 #record the total transactions in one block
        if 'Number of Transactions' in t:
            span_tag1=t.parent
            next_a_tag1=span_tag1.findNext('span')
            next_a_tag11=next_a_tag1.findNext('span')
            b_blocktransaction=int(next_a_tag11.contents[0].replace(',',''))

            total_page_temp=b_blocktransaction/5
            total_page_temp=int(total_page_temp)
            total_page=total_page_temp+1 #the total pages which record all transactions

            for i in range(1,total_page+1):  
                url_blockdetails=url_block+'?page='+str(i)
                htmltextblockdetails = scraper.get(url_blockdetails)
                if htmltextblockdetails.status_code == 200:
                    soup_objectblockdetails = BeautifulSoup(htmltextblockdetails.content, 'html.parser')
                for t1 in soup_objectblockdetails.find_all('span'):
                    if 'Hash' in t1:
                        span_tag2=t1.parent
                        next_a_tag22=span_tag2.findNext('a')
                        next_a_tag22_temp=next_a_tag22.get('href')
                        if '/address/' not in next_a_tag22_temp:
                            next_a_tag22_temp=next_a_tag22_temp.replace('/btc/tx/','')
                            b_id.append(next_a_tag22_temp)
                            transaction_id=next_a_tag22_temp
                            b_pool.append(mining_pool)
                            b_blockfinal.append(k)
                            
                            flag=0
                            url_iddetails="https://www.blockchain.com/btc/tx/"+transaction_id
                            htmltextid = scraper.get(url_iddetails)
                            if htmltextid.status_code == 200:
                                soup_objectid = BeautifulSoup(htmltextid.content, 'html.parser')
                            b_sourcetemp=[]
                            b_destinationtemp=[]

                            htmltextiddetails = scraper.get(url_iddetails)
                            if htmltextiddetails.status_code == 200:
                                soup_objectiddetails = BeautifulSoup(htmltextiddetails.content, 'html.parser')
                            for t2 in soup_objectiddetails.find_all('div'):
                                t2_class=t2.get('class')  
                                if t2_class:
                                    if 'gPBsWw' in t2_class: #find specifc class which could locate source address
                                        if 'COINBASE (Newly generated coins)' in t2.getText() :
                                            b_source.append('COINBASE (Newly generated coins)')
                                            b_sourcetemp.append('COINBASE (Newly generated coins)')
                                            b_sourceprice.append('/')
                                            flag=1

                                        else : 
                                            t2_a=t2.find('a')
                                            t2_href=t2_a.get('href')
                                            source_address=t2_href.replace('/btc/address/','')
                                            b_source.append(source_address)
                                            b_sourcetemp.append(source_address)

                                    if 'lhmncg' in t2_class: #find specifc class which could locate source address
                                        t2_a=t2.find('a')
                                        if t2_a:
                                            t2_href=t2_a.get('href')
                                            destination_address=t2_href.replace('/btc/address/','')
                                            b_destination.append(destination_address)
                                            b_destinationtemp.append(destination_address)

                                    if 'hdwAPC' in t2_class and flag==0:
                                        t2_span=t2.find('span')
                                        if t2_span:
                                            sourceprice=t2_span.getText()
                                            b_sourceprice.append(sourceprice)

                                    if 'cspEAW' in t2_class:
                                        t2_span=t2.find('span')
                                        if t2_span:
                                            destinationprice=t2_span.getText()
                                            b_destinationprice.append(destinationprice)
                                    if 'kKDjc' in t2_class:
                                        t2_span=t2.find('span')
                                        if t2_span:
                                            time=t2_span.getText()
                                            b_timestamp.append(time)

                            source_len=len(b_sourcetemp)
                            destination_len=len(b_destinationtemp)
                            if source_len > destination_len:
                                gap=source_len-destination_len
                                for i in range(1,gap+1):
                                    b_destination.append('')
                                    b_destinationprice.append('')
                                gap1=source_len-1
                                for i in range(1,gap1+1):
                                    b_id.append('')
                                    b_pool.append('')
                                    b_blockfinal.append('')
                                    b_timestamp.append('')
                            elif destination_len > source_len:
                                gap=destination_len-source_len
                                for i in range(1,gap+1):
                                    b_source.append('')
                                    b_sourceprice.append('')
                                gap1=destination_len-1
                                for i in range(1,gap1+1):
                                    b_id.append('')
                                    b_pool.append('')
                                    b_blockfinal.append('')
                                    b_timestamp.append('')
                            elif destination_len == source_len:
                                for i in range(1,source_len):
                                    b_id.append('')
                                    b_pool.append('')
                                    b_blockfinal.append('')
                                    b_timestamp.append('')

In [9]:
import csv
whole_list=zip(b_blockfinal,b_pool,b_id,b_timestamp,b_source,b_sourceprice,b_destination,b_destinationprice)
with open('bitcoin_data1.csv', 'w') as fp:
    writer = csv.writer(fp, delimiter = ',')
    writer.writerow(["block","mining_pool","transaction_id" ,"transaction_timestamp", "source_address","paid_price","destination_address","received_price"])
    for row in whole_list:
        writer.writerow(row)